# NanoOrganizer — Full Demo

This notebook exercises **every** supported data type end-to-end:

1. Create a `DataOrganizer` and a `RunMetadata`.
2. Simulate and link data for all 9 types (UV-Vis, SAXS 1D, WAXS 1D, DLS, XAS, SAXS 2D, WAXS 2D, SEM).
3. Save, reload from disk, and produce visualisations for every type.

Run top-to-bottom with **Kernel → Restart & Run All**.

In [ ]:
%matplotlib inline
import tempfile, os
from pathlib import Path

from NanoOrganizer import (
    DataOrganizer,
    RunMetadata,
    ReactionParams,
    ChemicalSpec,
    save_time_series_to_csv,
    # 1-D simulators
    simulate_uvvis_time_series_data,
    simulate_saxs_time_series_data,
    simulate_waxs_time_series_data,
    simulate_dls_time_series_data,
    simulate_xas_time_series_data,
    # 2-D simulators
    simulate_saxs2d_time_series_data,
    simulate_waxs2d_time_series_data,
    # Image simulator
    create_fake_image_series,
)

## Step 1 — Set up the organizer and run metadata

In [ ]:
# Use a temporary directory so the notebook is self-contained
BASE_DIR = tempfile.mkdtemp(prefix="nanoorg_demo_")
print("Working directory:", BASE_DIR)

org = DataOrganizer(BASE_DIR)

meta = RunMetadata(
    project="Project_Cu2O",
    experiment="2024-10-25",
    run_id="Cu2O_Full_Demo_001",
    sample_id="Sample_Demo",
    reaction=ReactionParams(
        chemicals=[
            ChemicalSpec(name="Cu(NO3)2", concentration=10.0, volume_uL=5.0),
            ChemicalSpec(name="NaOH",     concentration=100.0, volume_uL=2.0),
        ],
        temperature_C=80.0,
        description="Cu2O nanoparticle growth — full demo",
    ),
    tags=["cu2o", "demo", "full"],
    notes="Generated by full_demo.ipynb",
)

run = org.create_run(meta)

## Step 2 — Simulate & link all data types

In [ ]:
# ── UV-Vis ──────────────────────────────────────────────
times, wls, absorbance = simulate_uvvis_time_series_data()
uvvis_dir = os.path.join(BASE_DIR, "uvvis")
uvvis_files = save_time_series_to_csv(
    uvvis_dir, "uvvis",
    times, wls, absorbance,
    x_name="wavelength", y_name="absorbance",
)
run.uvvis.link_data(uvvis_files, time_points=sorted(set(times)))

In [ ]:
# ── SAXS 1D ─────────────────────────────────────────────
times, q, intensity = simulate_saxs_time_series_data()
saxs_dir = os.path.join(BASE_DIR, "saxs")
saxs_files = save_time_series_to_csv(
    saxs_dir, "saxs",
    times, q, intensity,
    x_name="q", y_name="intensity",
)
run.saxs.link_data(saxs_files, time_points=sorted(set(times)))

In [ ]:
# ── WAXS 1D ─────────────────────────────────────────────
times, two_theta, intensity = simulate_waxs_time_series_data()
waxs_dir = os.path.join(BASE_DIR, "waxs")
waxs_files = save_time_series_to_csv(
    waxs_dir, "waxs",
    times, two_theta, intensity,
    x_name="two_theta", y_name="intensity",
)
run.waxs.link_data(waxs_files, time_points=sorted(set(times)))

In [ ]:
# ── DLS ──────────────────────────────────────────────────
times, diameters, intensity = simulate_dls_time_series_data()
dls_dir = os.path.join(BASE_DIR, "dls")
dls_files = save_time_series_to_csv(
    dls_dir, "dls",
    times, diameters, intensity,
    x_name="diameter_nm", y_name="intensity",
)
run.dls.link_data(dls_files, time_points=sorted(set(times)))

In [ ]:
# ── XAS ──────────────────────────────────────────────────
times, energy, absorption = simulate_xas_time_series_data()
xas_dir = os.path.join(BASE_DIR, "xas")
xas_files = save_time_series_to_csv(
    xas_dir, "xas",
    times, energy, absorption,
    x_name="energy_eV", y_name="absorption",
)
run.xas.link_data(xas_files, time_points=sorted(set(times)))

In [ ]:
# ── SAXS 2D ─────────────────────────────────────────────
saxs2d_dir = os.path.join(BASE_DIR, "saxs2d")
saxs2d_files, saxs2d_cal = simulate_saxs2d_time_series_data(saxs2d_dir)
run.saxs2d.link_data(
    saxs2d_files,
    time_points=[0, 30, 60, 120, 180, 300, 600],
    **saxs2d_cal,
)

In [ ]:
# ── WAXS 2D ─────────────────────────────────────────────
waxs2d_dir = os.path.join(BASE_DIR, "waxs2d")
waxs2d_files, waxs2d_cal = simulate_waxs2d_time_series_data(waxs2d_dir)
run.waxs2d.link_data(
    waxs2d_files,
    time_points=[0, 30, 60, 120, 180, 300, 600],
    **waxs2d_cal,
)

In [ ]:
# ── SEM images ───────────────────────────────────────────
sem_dir = os.path.join(BASE_DIR, "sem")
sem_files = create_fake_image_series(sem_dir, n_images=5, pattern="sem")
run.sem.link_data(sem_files)

In [ ]:
# ── Save organizer to disk ───────────────────────────────
org.save()

## Step 3 — Reload & visualize

Everything below uses a freshly loaded organizer to prove the round-trip.

In [ ]:
import matplotlib.pyplot as plt

org2 = DataOrganizer.load(BASE_DIR)
run2 = org2.get_run("Project_Cu2O/2024-10-25/Cu2O_Full_Demo_001")
print("Loaded run:", run2.metadata.run_id)

### UV-Vis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
run2.uvvis.plot(plot_type="spectrum",  ax=axes[0])
run2.uvvis.plot(plot_type="kinetics",  ax=axes[1])
run2.uvvis.plot(plot_type="heatmap",   ax=axes[2])
fig.tight_layout()
plt.show()

### SAXS 1D

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
run2.saxs.plot(plot_type="profile",   ax=axes[0])
run2.saxs.plot(plot_type="kinetics",  ax=axes[1])
run2.saxs.plot(plot_type="heatmap",   ax=axes[2])
fig.tight_layout()
plt.show()

### WAXS 1D

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
run2.waxs.plot(plot_type="pattern",   ax=axes[0])
run2.waxs.plot(plot_type="kinetics",  ax=axes[1])
run2.waxs.plot(plot_type="heatmap",   ax=axes[2])
fig.tight_layout()
plt.show()

### DLS

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
run2.dls.plot(plot_type="size_dist",  ax=axes[0])
run2.dls.plot(plot_type="kinetics",   ax=axes[1])
run2.dls.plot(plot_type="heatmap",    ax=axes[2])
fig.tight_layout()
plt.show()

### XAS

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
run2.xas.plot(plot_type="xanes",     ax=axes[0])
run2.xas.plot(plot_type="kinetics",  ax=axes[1])
run2.xas.plot(plot_type="heatmap",   ax=axes[2])
fig.tight_layout()
plt.show()

### SAXS 2D

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
run2.saxs2d.plot(plot_type="detector",  ax=axes[0])
run2.saxs2d.plot(plot_type="azimuthal", ax=axes[1])
fig.tight_layout()
plt.show()

### WAXS 2D

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
run2.waxs2d.plot(plot_type="detector",  ax=axes[0])
run2.waxs2d.plot(plot_type="azimuthal", ax=axes[1])
fig.tight_layout()
plt.show()

### SEM image

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
run2.sem.plot(index=0, ax=ax)
plt.show()

---

All 9 data types loaded, saved, reloaded, and plotted successfully.